# Domain‑Adaptive Pretraining (DAPT)
This notebook demonstrates how to continue masked‑language model pretraining for **DeBERTa‑v3‑large** on an unlabeled news corpus.

In [ ]:
# Install dependencies (Colab/first run only)
!pip install -q transformers datasets accelerate peft

In [ ]:
# (Optional) Mount Google Drive to save checkpoints
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from src.pretrain_lm import run_dapt

run_dapt(
    model_name='microsoft/deberta-v3-large',
    data_file='data/external/unlabeled.txt',
    output_dir='outputs/dapt_checkpoints/',
    num_train_epochs=3,
    batch_size=4,  # Smaller batch for Colab
    learning_rate=5e-5,
)

After training completes, the adapted checkpoint is saved to **`outputs/dapt_checkpoints/`**. You can now fine‑tune this checkpoint with LoRA by setting `model_name_or_path` to that directory in your training script.